In [3]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
from vlnce_baselines.config.default import get_config
from vlnce_baselines import utils




/home/rafi/miniconda3/envs/vlnce/lib/python3.6/site-packages/gym/core.py:26: UserWarning: WARN: Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"
/home/rafi/PycharmProjects/habitat-lab/habitat/utils/visualizations/maps.py:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  "100x100.png",


In [54]:
import os

def list_best_result_all(result_dir, split, criteria, transformer_type=["normal", "enhanced", "full"], eval_dir="evals", col_ordering = ["path_length", "distance_to_goal", "ndtw", "oracle_success", "success", "spl"], allowed_split=["val_seen", "val_unseen"], sort_reverse=True):
    res_dict = {}
    keep_n_best = 1
    for upper_dir in transformer_type:
        path = os.path.join(result_dir, upper_dir)
        if os.path.exists(path):
            l = [d for d in os.listdir(path) if not d.startswith(".") and not d.startswith("_")]
            if len(l) > 0:
                for model in l:
                    model_result_dir = os.path.join(path, model, eval_dir)
                    if os.path.exists(model_result_dir):
                        res_dict[model_result_dir] = {}
                        result_files = utils.get_result_files_per_datasplit(model_result_dir)
                        result_table = utils.read_results_per_split(result_files)
                        if result_table is not None:
                            res_dict[model_result_dir] = result_table

    best_score = 0.0
    best_model = "No model found"
    all_best = {}
    for model_result_dir in res_dict.keys():
        if split in res_dict[model_result_dir].keys():
            _, frame = res_dict[model_result_dir][split]
            best_index = frame[criteria].nlargest(keep_n_best)
            current_res = frame[criteria][best_index.index]
            metrics = current_res.values[0]

            all_res = {s:res_dict[model_result_dir][s][1].loc[best_index.index].reindex(columns=col_ordering) for s in res_dict[model_result_dir].keys() if s in allowed_split}
            all_res = pd.concat(all_res, axis=1)
            all_best[model_result_dir] = metrics, best_index.index.values[0], all_res
            if metrics >= best_score:
                best_score = metrics
                best_model = model_result_dir

    print("Best:", best_model, split, best_score)
    return dict(sorted(all_best.items(), key=lambda item: item[1][0], reverse=sort_reverse))

# https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
# https://stackoverflow.com/questions/18528533/pretty-printing-a-pandas-dataframe

def get_report_config(config_filepath, epoch):
    
    conf_node = get_config(config_filepath)

    # Workaround I had a bug where the extraction of the epoch from the file name was buggy
    # as my regex was not good enough...
    # just retaining the last 2 digits 
    if epoch > 1000: 
        epoch = int(str(epoch)[-2])
        
    
    model_conf = conf_node.MODEL.DECISION_TRANSFORMER
    training_conf = conf_node.IL
    
    default = "_"
    default_yes = "y"
    default_no = "n"
    default_split = "train"
    instruction_encoding = "trans"
    AUG = "AUG"
    instruction_n_head = default
    instruction_n_layer = default
    instruction_dim = default
    
    net = model_conf.net
    
    if net == 'DecisionTransformerNet':
        net = "DT"
    if net == 'DecisionTransformerEnhancedNet':
        net = "E-DT"
    if net == 'FullDecisionTransformerNet':
        net = "F-DT"
    if model_conf.net in ['DecisionTransformerNet', 'DecisionTransformerEnhancedNet']:
        if model_conf.use_transformer_encoded_instruction:
            instruction_n_head = conf_node.MODEL.DECISION_TRANSFORMER.ENCODER.n_head
            instruction_n_layer = conf_node.MODEL.DECISION_TRANSFORMER.ENCODER.n_layer
            instruction_dim = conf_node.MODEL.DECISION_TRANSFORMER.hidden_dim
        else:
            instruction_encoding = "lstm"
            instruction_dim = conf_node.MODEL.INSTRUCTION_ENCODER.hidden_size
        
    else:
        instruction_n_head = conf_node.MODEL.DECISION_TRANSFORMER.n_head
        instruction_n_layer = conf_node.MODEL.DECISION_TRANSFORMER.n_layer
        instruction_dim = conf_node.MODEL.DECISION_TRANSFORMER.hidden_dim

    reward_type = model_conf.reward_type.split("_to_go")[0].upper()
    # work around for early inconsistencies in the project
    if reward_type == "POINT_NAV_REWARD":
        reward_type = "POINT_GOAL_NAV_REWARD"
    display_reward = ""
    if "SPARSE" in reward_type:
        display_reward = "SP"
    elif "NDTW" in reward_type:
        display_reward = "ND"
    else:
        display_reward = "PG"
    #display_reward = " ".join(reward_type.split("_REWARD")[0].split("_"))


    use_dagger = default
    dagger_iterations = default
    dagger_update_size = default
    dagger_p = default
    dagger_oa = default
    dagger_pe = default

    if not training_conf.DAGGER.preload_lmdb_features and training_conf.DAGGER.update_size != 10819:
        use_dagger = default_yes
        dagger_iterations = training_conf.DAGGER.iterations
        dagger_update_size = training_conf.DAGGER.update_size
        if training_conf.DECISION_TRANSFORMER.use_oracle_actions:
            dagger_oa = default_yes
        else:
            dagger_oa = default_no
        if training_conf.DECISION_TRANSFORMER.use_perfect_episode_only_for_dagger:
            dagger_pe = default_yes
        else:
            dagger_pe = default_no
        
        dagger_p = training_conf.DAGGER.p
        
        

    pretraining = default
    pretraining_name = default    
    if training_conf.load_from_ckpt:
        pretraining = default_yes
        pretraining_name = training_conf.ckpt_to_load.split("/ckpt")[0].split("/")[-1]

    main_task = conf_node.BASE_TASK_CONFIG_PATH.split("/")[-1].split(".")[0]
    env_drop = default

    if pretraining == default_yes:
        if  AUG.lower() in pretraining_name:
            aug_pretrained = AUG
            if "5epochs" in pretraining_name:
                aug_pretrained = aug_pretrained + " (5 epo.)"
            pretraining_name = aug_pretrained
        elif "50" in pretraining_name:
            pretraining_name = "train 50"
        else:
            pretraining_name = default_split
    train_split = default_split      
    if AUG.lower() in main_task:
        train_split = AUG
    if pretraining_name == "train 50":
        train_split = pretraining_name+"+"
    
    
    """
    displayed_conf = {"Model": net,
                  "Batch Size": training_conf.batch_size,
    "GPT Heads": model_conf.n_head,
    "GPT Layer": model_conf.n_layer,
    "GPT Dim": model_conf.hidden_dim,
                 "Instruction Encoding": instruction_encoding,
                 "Instruction Heads": instruction_n_head,
                 "Instruction Layers": instruction_n_layer,
                 "Instruction Dim": instruction_dim,
                 "Reward Type": display_reward,
                 "Reward Step Penalty": training_conf.DECISION_TRANSFORMER[reward_type].step_penalty,
                 "Reward Success":  training_conf.DECISION_TRANSFORMER[reward_type].success,
                 "DAGGER": use_dagger,
                 "DAGGER Iterations": dagger_iterations,
                 "DAGGER Update Size": dagger_update_size,
                 "DAGGER Oracle Actions": dagger_oa,
                 "DAGGER Perfect Episodes": dagger_pe,
                 "AUG": env_drop,
                 "Pre-training": pretraining,
                 "Pre-training Config": pretraining_name
                     }
    """
    
    if (dagger_iterations, dagger_update_size,dagger_p) == ("_","_","_"):
        da_it_up_p = default
    else:
        da_it_up_p = f"{dagger_iterations}, {dagger_update_size}, {dagger_p}"
    
    if dagger_oa == "_" and dagger_pe == "_":
        oa_pe = default
    else:
        oa_pe = f"{dagger_oa}, {dagger_pe}"
    
    displayed_conf = {"Agent": net,#config_filepath.split("/")[-1], #net
                  "Batch": training_conf.batch_size,
    "GPT H,L,D": f"{model_conf.n_head}, {model_conf.n_layer}, {model_conf.hidden_dim}",
                 "Inst. Enc.": instruction_encoding,
                 "Inst. H,L,D": f"{instruction_n_head}, {instruction_n_layer}, {instruction_dim}",
                 "Reward Type, Step Penalty, Success": f"{display_reward}, {training_conf.DECISION_TRANSFORMER[reward_type].step_penalty}, {training_conf.DECISION_TRANSFORMER[reward_type].success}",
                 #"Reward Step Penalty": training_conf.DECISION_TRANSFORMER[reward_type].step_penalty,
                 #"Reward Success":  training_conf.DECISION_TRANSFORMER[reward_type].success,
                 "DA": use_dagger,
                 "DA it., up., prob.": da_it_up_p,
                 "DA OA,PE": oa_pe,
                 "AUG": env_drop,
                 "Pre-training": pretraining,
                 "Comments": pretraining_name
                     }



    default_subcolumn = "-"
    latex_conf = {("Agent",default_subcolumn): net,
                  ("Batch", default_subcolumn): training_conf.batch_size,
                  ("GPT","H"): model_conf.n_head,
                  ("GPT","L"): model_conf.n_layer,
                ("GPT","D"): model_conf.hidden_dim,
                 ("Instruction", "Enc."): instruction_encoding,
                 ("Instruction", "H"): instruction_n_head,
                 ("Instruction", "L"): instruction_n_layer,
                 ("Instruction", "D"): instruction_dim,
                 ("Reward","Type") : display_reward,
                 ("Reward","Step"): training_conf.DECISION_TRANSFORMER[reward_type].step_penalty,
                 ("Reward","Success"):  training_conf.DECISION_TRANSFORMER[reward_type].success,
                 ("DAGGER","Prob.") : dagger_p,
                  ("DAGGER","It."): dagger_iterations,
                 ("DAGGER","Up."): dagger_update_size,
                 ("DAGGER","OA"): dagger_oa,
                 ("DAGGER","PE"): dagger_pe,
                 ("Train Split","Pre"): pretraining_name,
                 ("Train Split", "Current"): train_split,
                  ("Epoch", ""): epoch
                     }   


    return latex_conf


def get_config_and_results_for_print(all_best_res):

    it = len(all_best_res)
    print(f"\n################  {it}  results retrieved for {split} #################\n")
    report_conf_list = []
    report_result_list = []
    id = 1
    config_col_name = "#"
    model_col_name = "Agent"
    
    for k, v in all_best_res.items():
        _, epoch,result = v
        config_file_name = k.split(main_folder + "/")[1].split("/evals")[0].split("/")[1]       
        config_filepath = os.path.join(config_dir, config_file_name+".yaml")
        if os.path.exists(config_filepath):
            report_conf = get_report_config(config_filepath, epoch.item())
        else:
            print(f"Config not found: {config_filepath}")
            continue
        report_conf_list.append(report_conf)
        result.insert(loc=0, column=config_col_name, value=id)
        name = "DT"
        config_file_name_lower = config_file_name.lower()
        if "full" in config_file_name_lower:
            name = "F-"+name
        elif "enhanced" in config_file_name_lower:
            name = "E-"+name
            

        result.insert(loc=1, column=model_col_name, value=name)
        result.insert(loc=2, column="temp", value= config_file_name)
        report_result_list.append(result)
        id += 1

    report_conf_list = pd.DataFrame.from_dict(report_conf_list)
    report_conf_list.insert(loc=0, column=config_col_name, value=report_conf_list.index + 1)
    ordered_col = report_result_list[0].columns
    report_result_list = pd.concat(report_result_list).reindex(columns=ordered_col)
    #report_conf_list = report_conf_list.rename(columns = {epoch_col:(epoch_col,"-")})
    return report_conf_list, report_result_list

def html_display(df):
    display(HTML(df.to_html(index=False)))

In [55]:
# https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns
# https://stackoverflow.com/questions/18528533/pretty-printing-a-pandas-dataframe
from IPython.display import display, HTML

transformer_type = ["normal", "enhanced",
                    "full"]  # ["normal", "enhanced", "full"]
split = "val_unseen"
# main_folder = "checkpoints"
main_folder = "decision_transformer"
# result_dir = "../data/" + main_folder
result_dir = "results/" + main_folder
criteria = "spl"

config_dir = "vlnce_baselines/config/r2r_baselines/decision_transformer/all/"
# TL NE nDTW OS SR SP => "path_length", "distance_to_goal", "ndtw", "oracle_success", "success", "spl"
# "distance_to_goal", " success", "spl",  "ndtw", "path_length" , "oracle_success", "steps_taken"
#
all_best_res = list_best_result_all(result_dir, split, criteria,
                                    transformer_type)

report_conf_list, report_result_list = get_config_and_results_for_print(
    all_best_res)

Best: results/decision_transformer/normal/dt_08_d512_l8_h16_sparse_reward_to_go_dagger_it10_ep12_p0_75_pof_oat/evals val_unseen 0.20912843681619592

################  150  results retrieved for val_unseen #################

Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/full_08_sparse_reward_to_go_enc_l6_h16.yaml
Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/full_08_d512_sparse_reward_to_go_enc_l8_h16.yaml
Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/dt_08_d1024_l12_h16.yaml
Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/dt_08_sparse_reward_to_go_dagger_it06_ep30_p0.75_d128.yaml.yaml
Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/full_08_d512_sparse_reward_to_go_enc_l10_h16.yaml
Config not found: vlnce_baselines/config/r2r_baselines/decision_transformer/all/full_sv_b16_d512_state_instruct.yaml
Config not found: vlnce_bas

## Results.

The results are ordered from best to worst. The detailed corresponding configuration for each model is shown after the result table.

DT stands for Decision Transformer

F-DT stands for Full Decision Transformer

E-DT stands for Enhanced Decision Transformer

In [289]:
res = report_result_list.round(2)



res = res.rename(columns = {'val_seen':"Val Seen", "":"-", "path_length": "TL", "distance_to_goal": "NE", 'val_unseen':"Val Unseen", "ndtw":"nDTW", "oracle_success":"OS", "success":"SR", "spl":"SPL"})

res = res.iloc[0:, :]
res_2 = res.drop(columns="temp")
html_display(res_2)

c = "List of all experiments' results, ranked by descending SPL on the Validation Unseen data split. The rank in column \\# is also used as a look up id in table \\ref{tab:all-configs-final} to link the corresponding training configuration. \
\\newline The agents are based on Decision Transformer ('DT'), Enhanced Decision Transformer ('E-DT') or Full Decision Transformer ('F-DT').\
\\newline 'TL' stands for Trajectory Length, 'NE' stands for Navigation Error, 'nDTW' stands for normalized Dynamic Time Warping, 'SR' stands for success rate and 'SPL' stands for  Success weighted by (normalized inverse) Path Length."

# \cmidrule(l){3-8}\cmidrule(l){9-14}
res_2.to_latex(buf="./latex/res_list.tex", index=False, column_format= "@{\\hskip3pt}c"*len(res.columns), longtable=True, caption=c)



/home/rafi/miniconda3/envs/vlnce/lib/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## Corresponding Model Parameters

DT stands for Decision Transformer

F-DT stands for Full Decision Transformer

E-DT stands for Enhanced Decision Transformer

(H,L,D) stands for Head, Layers, Dimension
Inst. Enc. stands for Instrunction Encoder: values are either trans (transformer) or lstm
\_ is used when something is not applicable (for example when the Instruction encoder is lstm, the value may show (\_,\_, 128), as heads are not applicable for lstm)

In [231]:

d = report_conf_list.rename(columns = {'#':('#',"-")})

d = d.iloc[0:, :]
header, subcol = [list(a) for a in zip(*d.columns)]

subcol = [ s if s is not '' else '-' for s in subcol ]

d.columns= [header, subcol]
d.columns

MultiIndex([(          '#',       '-'),
            (      'Agent',       '-'),
            (      'Batch',       '-'),
            (        'GPT',       'H'),
            (        'GPT',       'L'),
            (        'GPT',       'D'),
            ('Instruction',    'Enc.'),
            ('Instruction',       'H'),
            ('Instruction',       'L'),
            ('Instruction',       'D'),
            (     'Reward',    'Type'),
            (     'Reward',    'Step'),
            (     'Reward', 'Success'),
            (     'DAGGER',   'Prob.'),
            (     'DAGGER',     'It.'),
            (     'DAGGER',     'Up.'),
            (     'DAGGER',      'OA'),
            (     'DAGGER',      'PE'),
            ('Train Split',     'Pre'),
            ('Train Split', 'Current'),
            (      'Epoch',       '-')],
           )

In [24]:
html_display(d)


In [290]:
c = "List of experiments' configurations. \
\\newline The agents are based on Decision Transformer ('DT'), Enhanced Decision Transformer ('E-DT') or Full Decision Transformer ('F-DT').\
\\newline 'H', 'L', 'D' stands for Heads, Layers and Dimensions. \
\\newline Concerning the Instruction section: 'Enc.'' stands for Encoder. It can be either a LSTM ('lstm') or a Tranformer ('transf'). In case of LSTM, the 'H' and 'L' columns are empty. \
\\newline Concerning the Reward section: The 3 types are Sparse Reward To Go ('SP'), Point Goal Navigation To Go ('PG') and nDTW to Go ('ND'). \
\\newline Concerning the DAGGER section: 'Prob.'' stands for probability, 'It.'' stands for iteration. Up. stands for Update Size. OA stands for Oracle Actions (whether the teacher forcing relies on the oracle's or agent's prediction). PE stands for Perfect Episodes (where unsuccessful episodes are discarded for DAGGER). )\
\\newline Concerning the Training Split section: 'Pre' stands for Pre-Training. 'AUG' refers to augmented data (e.g. training and EnvDrop split together). The last column, 'Epoch' shows the training epoch for the best model."
d.to_latex(buf="./latex/config_list.tex", index=False, column_format= "@{\\hskip3pt}c"*len(d.columns), longtable=True, caption=c)

#   \# & Model & Batch & \multicolumn{3}{c}{GPT} & \multicolumn{4}{c}{Instruction} & \multicolumn{3}{c}{Reward} & \multicolumn{5}{c}{DAGGER} & \multicolumn{2}{c}{Train Split} & Epoch \\
#   \cmidrule(l){4-6} \cmidrule(l){7-10} \cmidrule(l){11-13} \cmidrule(l){14-18} \cmidrule(l){19-20}


In [357]:
experiment_paths = "vlnce_baselines/config/r2r_baselines/decision_transformer/report_results"
experiments = ['normalized_visual_features',
 'inflection_weights',
 'action_for_final_pred',
 'layers_heads',
 'full_dt_ablation',
 'dagger_envdrop',
 'dim_test',
 'lr',
 'batch_test',
 'failed_to_converge',
 'reward',
 'env_drop',
 'rezero',
 'spatial_features',
 'dagger',
 'split_training_50',
 'instruction_encoding']

experiments = None

def get_experiment_dict(experiment_paths:str, experiments:list =None, dt_list = ["normal", "full", "enhanced"]):

    #os.listdir(experiment_paths)

    if experiments is None:
        experiments = os.listdir(experiment_paths)

    experiment_dict = {}

    def add_file_name(experiment_dict, result_dir, k, pref=""):
        if os.path.isdir(result_dir):
            for file in sorted(os.listdir(result_dir)):
                if file.endswith(".yaml") or file.endswith(".yml"):
                    experiment_dict[k].append( pref + file.split(".y")[0])

    for e in experiments:
        exp_dir = os.path.join(experiment_paths,e)
        if os.path.isdir(exp_dir):
            experiment_dict[e] = []
            add_file_name(experiment_dict, exp_dir, e)
            for dt in dt_list:
                add_file_name(experiment_dict, os.path.join(exp_dir,dt), e)
    
    return experiment_dict

experiment_dict = get_experiment_dict(experiment_paths, experiments)
print(len(experiment_dict))


#[ re.split( '/[a-z]*/', k.split(result_dir)[1].split("/evals")[0])[1] for k in all_best_res.keys() ]

18


In [366]:
"""
Allows to create latex tables as listed in the experiments directories.
"""
def create_experiment_results(res_tab, search, experiment_name):
    
    res = res_tab.copy().sort_values([("Agent","-"), ("#","-")])
    search = [s for s in search if sum(res["temp"]["-"] == s) > 0 ]# only keep experiments with results...
    if len(search) == 0:
        return None
    s = [res["temp"]["-"] == s for s in search]
    summation = s[0]
    for i in range(1, len(s)):
        summation += s[i]
    print()
    rows = list(summation)
    #return res[rows]

    da_train = [ "" if e == "_" else "DA" for e in d[rows]["DAGGER"]["Prob."]  ]
    pre_train = [ "" if e  == "_" else f"({e})*" for e in d[rows]["Train Split"]["Pre"] ]
    train = [ "" if e == "_" else e for e in d[rows]["Train Split"]["Current"]  ]
    #conf_ids = [ "See table \ref{tab:all-configs-final} with config \# " + str(id) for id in d[rows]["#"]["-"]]

    zipped = zip(pre_train, train,da_train )

    zipped = [(list(e)) for e in zipped]

    zipped_2 = [ ",".join([e for e in l if e != '']) for l in zipped ]
    #zipped_2  = [ ".".join(e) for e in zip(zipped, conf_ids ) ]

    for s, z in zip(search ,zipped_2):
        res.replace(s, z, inplace=True)
    
    res = res.rename(columns = {'temp':"Training Details"})
    res[rows].to_latex(buf=f"./latex/{experiment_name}.tex", index=False, column_format= "@{\\hskip3pt}c"*len(res.columns), longtable=True, caption=c, escape=False)

    return res[rows]

for k, v in experiment_dict.items():
    #enhanced = [ exp for exp in v if "enhanced" in exp or "full" in exp] 
    r = create_experiment_results(res, v, k)
